In [2]:
import collections
import urllib.request
import json
import requests
import csv
import os
import shutil
import collections

In [3]:
url_tissue_rui = "https://ccf-api.hubmapconsortium.org/v1/hubmap/rui_locations.jsonld"
url_reference_organ_data = "https://hubmapconsortium.github.io/ccf-ui/rui/assets/reference-organ-data.json"
url_collision_detection = "http://192.168.1.100:8080/get-collisions"

Step 1: save all the hubmap datasets in one folder

In [4]:
def copy_csv_files_in_one_folder(input_dir, output_dir):

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for folder in os.listdir(input_dir):
        full_folder = os.path.join(input_dir, folder)
        if os.path.isdir(full_folder):
            for file_name in os.listdir(full_folder):
                if file_name.endswith('.csv'):
                    abs_file_path = os.path.join(full_folder, file_name)
                    output_file_path = os.path.join(output_dir, file_name)

                    if folder.find('CxG') >= 0:
                        index = folder.find('CxG')
                        organ = folder[:index-1]
                        new_file_name = file_name[:-4] + '_' + organ + '.csv'
                        output_file_path = os.path.join(output_dir, new_file_name)

                    shutil.copy(abs_file_path, output_file_path)

In [5]:
input_dir = 'C:/Users/catherine/Desktop/research/hubmap/tissue-bar-graphs/csv'
output_dir = '../all_csv_2'
copy_csv_files_in_one_folder(input_dir, output_dir)

Step 2: for every tissue block, send a request and return the intersected volume and volume of AS.

In [4]:
file_mapping_rui_uuid_hubmap_id = './csv/mapping_rui_uuid_hubmap_id.csv'
file_mapping_rui_hubmap_id = './csv/mapping_rui_hubmap_id.csv'
hubmap_csv_folder = output_dir

In [8]:
mapping_rui_dataset_id = collections.defaultdict(list)

with open(file_mapping_rui_uuid_hubmap_id, 'r', encoding='UTF8') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        hubmap_id, uuid, rui = row
        mapping_rui_dataset_id[rui].append(hubmap_id)

with open(file_mapping_rui_hubmap_id, 'w', encoding='UTF8', newline="") as f:
    csv_writer = csv.writer(f)
    for rui, hubmap_ids in mapping_rui_dataset_id.items():
        csv_writer.writerow([rui] + hubmap_ids)

In [46]:
mapping_AS_cell_type_count = collections.defaultdict(dict)

with open(file_mapping_rui_hubmap_id, 'r', encoding='UTF8') as f:
    csv_reader = csv.reader(f)
    i = 0
    for row in csv_reader:
        if i == 0:
            i = 1
            continue

        rui_url = row[0]
        hubmap_ids = row[1:]

        dic_cell_type_count = collections.defaultdict(list)
        dic_cell_type_avg_count = collections.Counter()
        for hubmap_id in hubmap_ids:
            csv_path = os.path.join(hubmap_csv_folder, hubmap_id + '.csv')

            with open(csv_path, 'r', encoding='UTF8') as csv_file:
                dataset_csv_reader = csv.reader(csv_file)
                header = next(dataset_csv_reader)
                cell_type_index = header.index('cell_type')
                count_index = header.index('count')

                for item_cell in dataset_csv_reader:
                    dic_cell_type_count[item_cell[cell_type_index]].append(float(item_cell[count_index]))

        for cell, count_list in dic_cell_type_count.items():
            dic_cell_type_avg_count[cell] = sum(count_list) / len(hubmap_ids)

        get_response = requests.get(rui_url).json()
        rui_location = get_response['rui_location']
        post_response = requests.post(url=url_collision_detection, json=rui_location)
        target = rui_location['placement']['target']
        try:
            post_response = post_response.json()

            for AS_item in post_response:
                percentage_of_AS = AS_item['percentage_of_AS']
                percentage_of_tissue_block = AS_item['percentage_of_tissue_block']
                organ = AS_item['organ']
                node_name = AS_item['node_name']

                # if 'Skin' in organ:
                #     print(organ, hubmap_ids)

                for cell, avg_count in dic_cell_type_avg_count.items():
                    # initialize
                    if cell not in mapping_AS_cell_type_count[(organ, node_name)]:
                        mapping_AS_cell_type_count[(organ, node_name)][cell] = []

                    if 'Skin' in organ:
                        mapping_AS_cell_type_count[(organ, node_name)][cell].append(avg_count * percentage_of_tissue_block/ percentage_of_AS)
                    else:
                        mapping_AS_cell_type_count[(organ, node_name)][cell].append(avg_count / percentage_of_AS)
        except:
            pass

result = collections.defaultdict(dict)
for (organ, node_name) in mapping_AS_cell_type_count:
    for cell in mapping_AS_cell_type_count[(organ, node_name)]:
        cell_list = mapping_AS_cell_type_count[(organ, node_name)][cell]
        if 'Skin' in organ:
            print(organ, cell, cell_list)
        if len(cell_list) == 0:
            print(organ, node_name, cell)
        else:
            result[(organ, node_name)][cell] = int(sum(cell_list) / len(cell_list))

# print(mapping_AS_cell_type_count)
# print(result)

VH_M_Kidney_R VH_M_renal_column_R Afferent / Efferent Arteriole Endothelial
VH_M_Kidney_R VH_M_renal_column_R Ascending Thin Limb
VH_F_Skin CD68 
/ Macrophage [88466.59516537667, 70327.46847917805, 74976.07164909289]
VH_F_Skin T-Killer [24034.277299264184, 8625.066888955798, 46366.7811514127]
VH_F_Skin T-reg [80284.71353158461, 56394.6681200956, 62151.21728806384]
VH_F_Skin T-helper [669380.1911646131, 479686.41236269556, 1043745.8395360563]
VH_M_Skin CD68 
/ Macrophage [19888.80507396163, 229856.039523177, 62791.798876364584, 91007.33447535438, 297214.514681459]
VH_M_Skin T-Killer [6894.785758973365, 161355.89529441565, 26013.745248779614, 3914.293955929221, 58605.67895127361]
VH_M_Skin T-reg [2651.8406765282175, 537345.5758389502, 98672.82680571577, 58714.40933893831, 221864.3560298215]
VH_M_Skin T-helper [256963.3615555843, 2715650.1623135614, 740943.2267411021, 353265.02952261217, 2501206.655241856]
VH_F_Kidney_R VH_F_renal_column_R Afferent / Efferent Arteriole Endothelial


In [47]:
result

defaultdict(dict,
            {('VH_M_Kidney_R',
              'VH_M_outer_cortex_of_kidney_R'): {'Afferent / Efferent Arteriole Endothelial': 71974, 'Ascending Thin Limb': 5226589, 'Ascending Vasa Recta Endothelial': 421542, 'B': 15744, 'Classical Dendritic': 488119, 'Connecting Tubule': 3022323, 'Connecting Tubule Intercalated Type A': 27481, 'Connecting Tubule Principal': 664661, 'Cortical Collecting Duct Intercalated Type A': 1154457, 'Cortical Collecting Duct Principal': 1037744, 'Cortical Thick Ascending Limb': 10098949, 'Cortical Vascular Smooth Muscle / Pericyte': 165173, 'Descending Thin Limb Type 1': 1502880, 'Descending Thin Limb Type 2': 2815519, 'Descending Thin Limb Type 3': 118921, 'Descending Vasa Recta Endothelial ': 152455, 'Distal Convoluted Tubule Type 1': 5864833, 'Distal Convoluted Tubule Type 2': 293828, 'Fibroblast': 2058312, 'Glomerular Capillary Endothelial': 492863, 'Inner Medullary Collecting Duct': 4769141, 'Intercalated Type B': 730093, 'Lymphatic Endothel

In [48]:
result[('VH_M_Skin', 'VH_M_skin')]

{'CD68 \n/ Macrophage': 140151,
 'T-Killer': 51356,
 'T-reg': 183849,
 'T-helper': 1313605}

In [49]:
file_cell_type_count_per_AS = './csv/cell_type_count_per_AS.csv'

with open(file_cell_type_count_per_AS, 'w', encoding='UTF-8', newline='') as f:
    csv_writer = csv.writer(f)
    for (organ, AS) in result:
        dic_cell_type_count = result[(organ, AS)]

        for cell_type, count in dic_cell_type_count.items():
            csv_writer.writerow([organ, AS, cell_type, count])

file_cell_type_count_per_AS_Skin = './csv/cell_type_count_per_AS_Skin.csv'
with open(file_cell_type_count_per_AS_Skin, 'w', encoding='UTF-8', newline='') as f:
    csv_writer = csv.writer(f)
    for (organ, AS) in result:
        if 'Skin' in organ:
            dic_cell_type_count = result[(organ, AS)]

            for cell_type, count in dic_cell_type_count.items():
                csv_writer.writerow([organ, AS, cell_type, count])

Create a table: Tissue block by HuBMAP IDs | AS it collides with (1) | AS it collides with (1) |...| AS it collides with (n)

In [6]:
file_mapping_rui_uuid_hubmap_id = './csv/mapping_rui_uuid_hubmap_id.csv'
file_hubmap_id_collision_AS = "./csv/hubmap_id_collision_AS.csv"

with open(file_mapping_rui_uuid_hubmap_id, 'r', encoding='UTF8') as f, open(file_hubmap_id_collision_AS, 'w', encoding='UTF-8', newline='') as f2:
    csv_reader = csv.reader(f)
    csv_writer = csv.writer(f2)
    csv_writer.writerow(['HubMAP ID', 'ASs'])
    i = 0

    for row in csv_reader:

        if i == 0:
            i = 1
            continue

        hubmap_id, uuid, rui_url = row
        print(hubmap_id)
        get_response = requests.get(rui_url).json()
        rui_location = get_response['rui_location']
        post_response = requests.post(url=url_collision_detection, json=rui_location)

        try:
            post_response = post_response.json()
            collision_AS = []
            for AS_item in post_response:
                # percentage_of_AS = AS_item['percentage_of_AS']
                # organ = AS_item['organ']
                node_name = AS_item['node_name']
                collision_AS.append(node_name)
            csv_writer.writerow([hubmap_id] + collision_AS)
        except:
            print(post_response.content)


HBM876.HTRX.668
HBM537.GVTL.683
HBM447.KCHB.778
HBM552.PBBF.363
HBM282.BKRZ.254
HBM834.THPC.928
HBM958.GZCX.985
HBM684.TLPQ.738
HBM476.VHDB.268
HBM393.DZDZ.972
HBM942.NXWW.245
HBM427.KFXP.323
HBM634.QQWF.789
HBM664.LQSH.693
HBM873.NSHG.559
HBM258.HQCL.349
HBM243.MTZV.726
HBM252.JLSW.848
HBM246.KBZS.276
HBM495.ZFDT.229
HBM242.HNLH.466
HBM844.KXXX.692
HBM979.KNQM.454
HBM525.LRPL.332
HBM332.KQWB.454
HBM688.FLJB.475
HBM347.MRTV.333
HBM996.XSNK.694
HBM669.GWVL.893
HBM399.TFTQ.282
HBM382.HNKL.447
HBM537.LTFK.379
HBM969.VNQL.553
HBM892.MXFF.848
HBM992.XNZH.647
HBM574.NFCS.842
HBM224.GLDC.549
HBM264.RWRW.668
HBM773.WCXC.264
HBM268.RGJT.587
HBM432.BPZM.698
HBM578.SRZG.769
HBM253.HFDZ.866
HBM894.PGJZ.258
HBM229.HKHH.537
HBM865.PBJW.845
HBM396.WPKL.954
HBM839.TQNM.958
HBM764.ZCQR.585
HBM797.PDPS.368
HBM982.TPNQ.737
HBM375.ZKZZ.765
HBM654.BKGL.942
HBM639.SGGN.469
HBM297.VBLS.984
